In [1]:
# For ChromeDriverManager
# pip install webdriver-manager

# For substring -need to type this into git, won't work here (at least for me)
# pip install substring

In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import substring

import os
import pickle
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # Useful for sending keys (not necessarily used below) #cyh
from selenium.webdriver.common.action_chains import ActionChains # Useful for queuing commands (not necessarily used below) #cyh
from webdriver_manager.chrome import ChromeDriverManager # requires pip install webdriver-manager

In [3]:
#Create data frame to write info to
paper_df = pd.DataFrame(index=range(10000), columns=['Source', 'Title', 'Abstract', 'Journal', 'Author', 'Date', 'Keyword', 'Keyword2', 'Keyword3', 'Keyword4', 'Keyword5', 'General', "Citation", "Field", "Search Term"])

In [4]:
# Change the URL topic here!
# I filter down to journal articles only. I also put the search term into quotation marks

#url= "https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%22&sort=relevance&fos=political-science"
url= "https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics"

# List of topics and their search term links
# International (selected for political science papers only)
# https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=%22international%22&sort=relevance&fos=political-science
# "Space" (selected for Physics papers only)
# https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics

In [5]:
# WEBSCRAPING CELL
# Better version if your computer is able to detect the chromedriver in the install folder; mine cannot #cyh
#chrome_driver = "/Applications/chromedriver"
#os.environ["webdriver.chrome.driver"] = chrome_driver
#driver = webdriver.Chrome(chrome_driver)
driver = webdriver.Chrome(ChromeDriverManager().install())

# This is the beginning URL of all papers on Semantics Scholar
url_base = "https://www.semanticscholar.org/paper/"

paper_counter = 0
paper_counter_temp = 0
# Loop through X Results Pages
max_page_results = 10


for i in range(1,max_page_results+1): # add 1 due to starting at index 1
    url_page= url + "&page=" + str(i)
    driver.get(url_page)
    print("i = " + str(i) + " " + driver.current_url)
    time.sleep(2)    

    # Get the paper titles
    search_results = driver.find_elements_by_class_name("search-result-title") 
    paper_counter_temp = paper_counter
    for search_result_counter in range(0, len(search_results)):
        paper_df['Title'][paper_counter_temp] = search_results[search_result_counter].text
        paper_counter_temp = paper_counter_temp + 1
    
    
    hyperlinks = driver.find_elements_by_tag_name("a")
    
    # Get the URLs to the papers
    paper_url_list = []
    for hyperlink_counter in range(0, len(hyperlinks)):
        hyperlink_url = hyperlinks[hyperlink_counter].get_attribute("href")
        # Check if this element had a URL
        if hyperlink_url is not None:
            # Paper urls begin with "https://www.semanticscholar.org/paper/" as defined in url_base
            if hyperlink_url[0:len(url_base)] == url_base:
                paper_url_list.append(hyperlink_url)
            
    
    print("Number of papers are " + str(len(paper_url_list)))
    # try next sibling as an option
    # better:  try find_all("a") and look for the href
    
    # Loop through each search result on the search results page
    for j in range(0, len(paper_url_list)):
        paper_url_list
        # "Click" on the link by going directly to the URL. Directly going avoids problems associated with clicking, such as
        # banners/etc covering the hyperlinks and inconsistent website behavior.
        
        driver.get(paper_url_list[j])
        
        # Locate the "Continue Reading" button to expand the abstract and click it.  If no such button found, then do nothing.
        try:
            clickable_more_button = driver.find_element_by_partial_link_text("CONTINUE READING")
            clickable_more_button.click()
        except:
            pass
        time.sleep(1)
        
        response = requests.get(driver.current_url)
        time.sleep(1)
        
        soup = BeautifulSoup(response.text, "lxml")
    
        citations= soup.find_all(class_="bibtex-citation")
        #abstracts= soup.find_all(class_= "text-truncator abstract__text text--preline")
        abstracts= soup.find_all(class_= "fresh-paper-detail-page__abstract")
        keywords= soup.find_all(class_ ="preview-box__target")
        generals= soup.find_all(class_="schema-data")
        
        # Paper Titles are added near the top of the Search Results page loop
        #if len(abstracts) > 0:
        #    paper_df['Abstract'][paper_counter] = abstracts[0].text
        if len(keywords) > 0:
            paper_df['Keyword'][paper_counter] = keywords[0].text
            if len(keywords) > 1:
                paper_df['Keyword2'][paper_counter] = keywords[1].text
                if len(keywords) > 2:
                    paper_df['Keyword3'][paper_counter] = keywords[2].text
                    if len(keywords) > 3:
                        paper_df['Keyword4'][paper_counter] = keywords[3].text
                        if len(keywords) > 4:
                            paper_df['Keyword5'][paper_counter] = keywords[4].text
        #This cleans and adds citation and general fields to dataframe
        citations= citations[0].text
        paper_df['Citation'][paper_counter] = "".join(s for s in citations if ord(s)>31 and ord(s)<126)
        generals= generals[0].text
        paper_df['General'][paper_counter] = "".join(s for s in generals if ord(s)>31 and ord(s)<126)

#This should clean the citation column to get date, authors, journal and abstract.
        citation= paper_df['Citation'][paper_counter]
        citation2= citation.split("}, ")
        
        general= paper_df['General'][paper_counter]
        general2 = general.split("description")
        general2= general2[2].split('{')
        
        for i in range(0, len(citation2)):
            if "author=" in citation2[i]:
                author= citation2[i]
                author= author.split("={")
                author= author[1]
            if "journal=" in citation2[i]:
                journal= citation2[i]
                journal= journal.split("={")
                journal= journal[1]
            if "year=" in citation2[i]:
                date= citation2[i]
                date= date.split("={")
                date= date[1]
                
        paper_df['Abstract'][paper_counter] = general2[0]
        paper_df['Author'][paper_counter] = author
        paper_df['Journal'][paper_counter] = journal
        paper_df['Date'][paper_counter] = date
        paper_df['Source'][paper_counter] = driver.current_url
        paper_counter = paper_counter + 1
        
        # Return to Search Results page
        driver.back()
        if driver.current_url != url_page:
            driver.get(url_page)
        
        time.sleep(1)
        
# Purposely write the results as they come rather than all at once
# paper_df.to_pickle("./paper_data.pkl")

paper_df.to_csv("Paper_data1.csv", index=False)


Checking for win32 chromedriver:77.0.3865.40 in cache
Driver found in C:\Users\602772\.wdm\chromedriver\77.0.3865.40\win32/chromedriver.exe
i = 1 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics&page=1
Number of papers are 0
i = 2 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics&page=2
Number of papers are 0
i = 3 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics&page=3
Number of papers are 0
i = 4 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics&page=4
Number of papers are 10
i = 5 https://www.semanticscholar.org/search?publicationType%5B0%5D=JournalArticle&q=Space&sort=relevance&fos=physics&page=5
Number of papers are 10


SSLError: HTTPSConnectionPool(host='www.semanticscholar.org', port=443): Max retries exceeded with url: /paper/Picturing-qubits-in-phase-space-Wootters/0b1a4b8d08232356c7c1dd6b4bc07bd34f728325 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)')))

In [6]:
#paper_df.to_csv("Paper_data1.csv", index=False)

In [ ]:
# This cell for testing access to web elements

#url_page= url + "&page=" + "2"
#driver = webdriver.Chrome(ChromeDriverManager().install())
#driver.get(url_page)
#time.sleep(2)
#soupResultsPage = BeautifulSoup(driver.page_source, "html.parser")
#search_results = driver.find_elements_by_class_name("search-result-title")
#search_results[1].click()
#time.sleep(1)

#driver.back()
#time.sleep(1)
#search_results = driver.find_elements_by_class_name("search-result-title")
#search_results[2].click()
